In [1]:
import os
import sys
import time
from datetime import datetime, timedelta
from glob import glob
from os.path import exists, getmtime, join, lexists, realpath
from pathlib import Path

import pandas as pd
import sqlalchemy as sa
import xdg
import yfinance as yf
from environs import Env
from loguru import logger
from sqlalchemy import create_engine, text

In [2]:
_run_ts = time.time()
_run_dt = datetime.fromtimestamp(_run_ts).astimezone()
_run_localtime = time.localtime(_run_ts)
_run_gmtime = time.gmtime(_run_ts)
_run_ymd = time.strftime("%Y%m%d", _run_localtime)
_run_hms = time.strftime("%H%M%S", _run_localtime)
_run_ymdhms = f"{_run_ymd}_{_run_hms}"
_iso_datefmt = "%Y-%m-%d %H:%M:%S%z"
_fdate = _run_dt.strftime("%Y-%m")

In [3]:
# date_from = datetime(1970, 1, 1).date()
# date_thru = (_run_dt - timedelta(days=1)).date()
date_from = (_run_dt - timedelta(days=41)).date()
date_thru = _run_dt.date()
tickers = ["BTC-CAD", "BTC-USD"]
ticker1 = "BTC-CAD"
ticker2 = "BTC-USD"

## Tickers

In [8]:
btccad = yf.Ticker("BTC-CAD")

In [10]:
btccad.info

{'name': 'Bitcoin',
 'startDate': 1278979200,
 'description': 'Bitcoin (BTC) is a cryptocurrency launched in 2010. Users are able to generate BTC through the process of mining. Bitcoin has a current supply of 19,795,412. The last known price of Bitcoin is 100,114.20666872 USD and is down -0.77 over the last 24 hours. It is currently trading on 11840 active market(s) with $68,581,175,228.25 traded over the last 24 hours. More information can be found at https://bitcoin.org/.',
 'maxAge': 86400,
 'priceHint': 2,
 'previousClose': 142204.36,
 'open': 142204.36,
 'dayLow': 141257.72,
 'dayHigh': 144325.89,
 'regularMarketPreviousClose': 142204.36,
 'regularMarketOpen': 142204.36,
 'regularMarketDayLow': 141257.72,
 'regularMarketDayHigh': 144325.89,
 'volume': 88577024000,
 'regularMarketVolume': 88577024000,
 'averageVolume': 69135012260,
 'averageVolume10days': 119976407813,
 'averageDailyVolume10Day': 119976407813,
 'marketCap': 2857009938432,
 'fiftyTwoWeekLow': 51927.13,
 'fiftyTwoWee

In [4]:
df1 = yf.download(ticker1, start=date_from, end=date_thru, interval="1d", multi_level_index=False, progress=False)
df1.drop("Adj Close", axis=1, inplace=True)
df1.index.name = df1.index.name.lower()
cols = [x.lower().replace(" ", "_") for x in df1.columns]
df1.columns = cols
df1 = df1[["open", "low", "high", "close", "volume",]]
df1.rename(columns={"low": "lo", "high": "hi"}, inplace=True)
cols = ["cad_" + x for x in df1.columns]
df1.columns = cols

df1["cad_range"] = df1.cad_hi - df1.cad_lo
df1["cad_prev"] = df1.cad_close.shift(1)
df1["cad_chg_amt"] = df1.cad_close - df1.cad_prev
df1["cad_chg_pct"] = (df1.cad_close - df1.cad_prev) * 100 / df1.cad_prev

df1.to_csv(f"btc-cad-daily_{_run_ymdhms}.csv", sep="\t")

In [5]:
df2 = yf.download(ticker2, start=date_from, end=date_thru, interval="1d", multi_level_index=False, progress=False)

df2.drop("Adj Close", axis=1, inplace=True)
df2.index.name = df2.index.name.lower()
cols = [x.lower().replace(" ", "_") for x in df2.columns]
df2.columns = cols
df2 = df2[["open", "low", "high", "close", "volume",]]
df2.rename(columns={"low": "lo", "high": "hi"}, inplace=True)
cols = ["usd_" + x for x in df2.columns]
df2.columns = cols

df2["usd_range"] = df2.usd_hi - df2.usd_lo
df2["usd_prev"] = df2.usd_close.shift(1)
df2["usd_chg_amt"] = df2.usd_close - df2.usd_prev
df2["usd_chg_pct"] = (df2.usd_close - df2.usd_prev) * 100 / df2.usd_prev

df2.to_csv(f"btc-usd-daily_{_run_ymdhms}.csv", sep="\t")

In [6]:
df = df1.join(df2)

In [7]:
df["date_id"] = pd.to_datetime(df.index)
df.index = df.index.strftime("%Y%m%d").astype(int)
df.index.name = "id"
df = df[['date_id',
    'cad_open', 'cad_lo', 'cad_hi', 'cad_close', 'cad_volume', 
    'cad_range', 'cad_prev', 'cad_chg_amt', 'cad_chg_pct', 
    'usd_open', 'usd_lo', 'usd_hi', 'usd_close', 'usd_volume', 
    'usd_range', 'usd_prev', 'usd_chg_amt', 'usd_chg_pct',]]
df.head()

,date_id,cad_open,cad_lo,cad_hi,cad_close,cad_volume,cad_range,cad_prev,cad_chg_amt,cad_chg_pct,usd_open,usd_lo,usd_hi,usd_close,usd_volume,usd_range,usd_prev,usd_chg_amt,usd_chg_pct
id,,,,,,,,,,,,,,,,,,,
20241102,2024-11-02,97846.898438,95813.632812,99567.218750,97042.343750,69817997748,3753.585938,NaN,NaN,NaN,69486.023438,69033.718750,69867.351562,69289.273438,18184612091,833.632812,NaN,NaN,NaN
20241103,2024-11-03,97047.304688,96422.851562,97587.226562,96779.796875,25399356939,1164.375000,97042.343750,-262.546875,-0.270549,69296.382812,67482.523438,69361.656250,68741.117188,34868307655,1879.132812,69289.273438,-548.156250,-0.791113
20241104,2024-11-04,96789.718750,94256.218750,96880.898438,95696.500000,48541182268,2624.679688,96779.796875,-1083.296875,-1.119342,68742.132812,66803.648438,69433.179688,67811.507812,41184819348,2629.531250,68741.117188,-929.609375,-1.352334
20241105,2024-11-05,95697.921875,92876.109375,96562.109375,94267.492188,57252664769,3686.000000,95696.500000,-1429.007812,-1.493271,67811.171875,67458.867188,70522.789062,69359.562500,46046889204,3063.921875,67811.507812,1548.054688,2.282879
20241106,2024-11-06,94267.023438,93803.781250,97701.070312,95956.601562,63704305620,3897.289062,94267.492188,1689.109375,1.791826,69358.500000,69322.031250,76460.156250,75639.078125,118592653963,7138.125000,69359.562500,6279.515625,9.053569
